In [8]:
import sys
import os

# Add the directory to sys.path
cellradar_path = os.path.abspath('//folder/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/github/cellradar/cellradar')
sys.path.append(cellradar_path)

In [2]:
pip install -e ..

Obtaining file:///Users/mcanela/Documents/github/cellradar
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached pathlib-1.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
  Building editable for cellradar (pyproject.toml) ... done
  Created wheel for cellradar: filename=cellradar-0.1.0-py3-none-any.whl size=3020 sha256=6070dabf848a51c755164d7c54187dd0b412e1825dc3b648e1cc31176848aaad
  Stored in directory: /private/var/folders/v6/ndr2p31s5g17kx20bdsmb9kw0000gn/T/pip-ephem-wheel-cache-y1bcv2r1/wheels/21/f6/17/f130d0acae5c2a45e9d035e9f0e2949fba809c3bd38638d567
  Created wheel for pickle5: filename=pickle5-0.0.11-

In [3]:
from cellradar.main import analyze

ModuleNotFoundError: No module named 'cellradar'

# Analyzing single markers

In [ ]:
from main import analyze
from pathlib import Path
import pickle as pkl

## tdTomato

In [9]:
tdt_folder = Path("//folder/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/github/cellradar/examples/sample_data/tdt")
with open("//folder/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/github/cellradar/examples/sample_data/model/best_model_svm.pkl", "rb") as file:
    best_model = pkl.load(file)

In [10]:
analyze(tdt_folder, 'Reds', best_model)

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


Applying prediction model: 100%|██████████| 137/137 [05:22<00:00,  2.35s/image]


## cFos

In [ ]:
cfos_folder = Path("//folder/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/github/cellradar/examples/sample_data/cfos")

In [ ]:
analyze(cfos_folder, 'Greens')

# Calculating colocalization

In [ ]:
from predicting import colocalize

processed_rois_path_1 = Path('C:/Users/mcanela/Desktop/sample_jose/cfos_analysis/rois_processed')
images_path_1 = Path('C:/Users/mcanela/Desktop/sample_jose/cfos')
processed_rois_path_2 = Path('C:/Users/mcanela/Desktop/sample_jose/tdt_analysis_with_model/rois_processed')
images_path_2 = Path('C:/Users/mcanela/Desktop/sample_jose/tdt')

In [ ]:
colocalize(processed_rois_path_1, images_path_1, processed_rois_path_2, images_path_2)

In [ ]:
# path = image_folder.parent / 'features.csv'
# with open(path, 'rb') as file:
#     best_model = pkl.load(file)
# df = pd.read_csv(path, index_col=None)
# df = df.drop(columns=['Unnamed: 0'])
# X = X_test
# y = y_test

In [ ]:
# df = pd.read_csv("C:/Users/mcanela/Desktop/sample_jose/tdt_analysis_with_model/counts.csv")
# split_df = df["file_name"].str.split("_", expand=True)
# df["id"] = split_df[0]
# grouped_df = df.groupby("id")["num_cells"].mean().reset_index()
# grouped_df.to_excel("C:/Users/mcanela/Desktop/sample_jose/counts_grouped.xlsx", index=False)


# split_df = df["file_name"].str.split("_", expand=True)
# df[["group", "id", "brain", "replica"]] = split_df
# grouped_df = df.groupby(["group", "id", "brain"])["cells_per_squared_mm"].mean().reset_index()
# grouped_df.columns = ["group", "id", "brain", "mean_cells_per_squared_mm"]
# grouped_df.to_excel(os.path.join(output_folder, "results_friendly.xlsx"), index=False)